In [1]:
import requests
import pandas as pd
import sqlite3
import time as tm
from datetime import datetime, timedelta
db_path = r'D:\Rfiles\量化投资学习\database\high_freq.db'

# 库表结构和数据类型

In [3]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# 遍历表名并获取每个表的列信息
for table in tables:
    table_name = table[0]
    print("Table: ", table_name)

    cursor.execute("SELECT COUNT(*) FROM {}".format(table_name))
    row_count = cursor.fetchone()[0]
    print("Row count: ", row_count)

    cursor.execute("PRAGMA table_info({})".format(table_name))
    columns = cursor.fetchall()
    for column in columns:
        column_name = column[1]
        column_type = column[2]
        print("Column: {}, Type: {}".format(column_name, column_type))

    print("\n")
conn.close()


Table:  Security_name
Row count:  5288
Column: THSCODE, Type: TEXT
Column: SECURITY_NAME, Type: TEXT


Table:  zhangtinggu
Row count:  7413111
Column: time, Type: TEXT
Column: thscode, Type: TEXT
Column: open, Type: REAL
Column: high, Type: REAL
Column: low, Type: REAL
Column: close, Type: REAL
Column: avgPrice, Type: REAL
Column: volume, Type: REAL
Column: amount, Type: REAL
Column: change, Type: REAL
Column: changeRatio, Type: REAL
Column: turnoverRatio, Type: REAL
Column: sellVolume, Type: REAL
Column: buyVolume, Type: REAL
Column: BBI, Type: REAL
Column: MA, Type: REAL
Column: MACD, Type: REAL
Column: TRIX, Type: REAL
Column: RSI, Type: REAL
Column: ARBR, Type: REAL
Column: BOLL, Type: REAL
Column: VOSC, Type: REAL
Column: ZDZB, Type: REAL


Table:  zhangtinggu_Factors
Row count:  24818
Column: thscode, Type: TEXT
Column: ths_stock_short_name_stock, Type: TEXT
Column: ths_controlling_holder_held_ratio_stock, Type: TEXT
Column: ths_the_sw_industry_index_code_stock, Type: TEXT
Column

In [10]:
conn = sqlite3.connect(db_path)

df = pd.read_sql("SELECT * FROM zhangtinggu_Factors", conn)
df = df.iloc[::-1]
df

,thscode,ths_stock_short_name_stock,ths_controlling_holder_held_ratio_stock,ths_the_sw_industry_index_code_stock,ths_bod_num_stock,ths_techcore_num_stock,ths_chairman_held_num_stock,ths_top10_hlolder_held_ratio_stock,ths_float_holder_held_ratio_stock,ths_accounts_payable_pit_stock,...,ths_mfi_stock,ths_ddi_stock,ths_macd_stock,ths_rsi_stock,ths_adtm_stock,ths_boll_stock,ths_dptb_stock,ths_mi_stock,ths_std_stock,ths_company_patent_quantity_stock
24817,603390.SH,通达电气,"33.3700,29.1200",801880,9,5.0,None,66.45,33.3717,1.185173e+08,...,70.895421,0.713710,0.439622,93.331442,0.838346,7.289608,0.285714,2.93,0.708179,175.0
24816,002173.SZ,创新医疗,17.2800,801150,9,NaN,None,42.78,17.2850,1.968014e+08,...,47.198291,-0.149068,-0.104597,56.539946,0.678082,7.505100,0.857143,-0.07,0.481414,20.0
24815,001270.SZ,铖昌科技,47.2200,801740,9,3.0,None,65.04,2.6764,8.490958e+06,...,64.129894,0.123060,0.520107,44.708385,0.525306,76.672686,1.000000,5.95,3.401834,21.0
24814,603123.SH,翠微股份,29.4600,801200,9,NaN,None,59.36,21.5455,4.156734e+08,...,34.818437,-0.307992,0.429268,26.670255,0.324826,11.529183,0.571429,-2.33,1.423435,NaN
24813,300029.SZ,ST天龙,12.7700,801720,5,NaN,None,39.54,12.7669,1.449271e+08,...,64.792990,0.081761,0.344963,64.084435,0.828269,5.929262,0.428571,0.80,0.702864,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,002406.SZ,远东传动,23.4500,801880,9,NaN,None,35.68,5.8632,2.405979e+08,...,46.669069,-0.378378,-0.085660,48.742127,0.642686,5.739873,0.571429,-1.15,0.580372,165.0
3,002045.SZ,国光电器,13.4700,801080,7,NaN,None,37.48,13.4739,8.207424e+08,...,54.510813,0.031609,0.496782,60.107165,0.781931,12.347805,0.714286,-0.55,1.060146,219.0
2,601633.SH,长城汽车,60.2000,801880,7,15.0,None,92.11,60.2028,2.589650e+10,...,55.597729,0.019608,0.777263,57.913245,0.685686,24.506672,0.571429,-1.01,1.951092,9936.0
1,600203.SH,福日电子,15.8900,801080,9,NaN,None,38.25,15.8914,8.825115e+08,...,82.833501,0.375912,0.544315,56.582067,0.781127,5.767175,0.714286,1.52,0.861213,3.0


In [11]:
df_sorted = df.sort_values(by=['thscode'], ascending=True)

# 按照索引进行排序
# df_sorted = df_sorted.sort_index(ascending=False)
df_sorted

,thscode,ths_stock_short_name_stock,ths_controlling_holder_held_ratio_stock,ths_the_sw_industry_index_code_stock,ths_bod_num_stock,ths_techcore_num_stock,ths_chairman_held_num_stock,ths_top10_hlolder_held_ratio_stock,ths_float_holder_held_ratio_stock,ths_accounts_payable_pit_stock,...,ths_mfi_stock,ths_ddi_stock,ths_macd_stock,ths_rsi_stock,ths_adtm_stock,ths_boll_stock,ths_dptb_stock,ths_mi_stock,ths_std_stock,ths_company_patent_quantity_stock
21144,000004.SZ,国华网安,18.2200,801750,5,NaN,None,38.26,18.2200,9.097724e+07,...,35.632925,-0.642129,-0.545291,23.047736,0.225541,15.338615,0.714286,-3.41,1.518449,1.0
19260,000004.SZ,国华网安,18.2200,801750,5,NaN,None,38.26,18.2200,9.097724e+07,...,35.460733,-0.341108,-0.567686,42.898841,0.380952,14.712631,0.857143,-1.20,1.665254,1.0
12666,000004.SZ,国华网安,18.2200,801750,5,NaN,None,38.26,18.2200,9.097724e+07,...,60.378845,-0.010189,-0.321770,60.094441,0.327869,15.117429,0.571429,-0.13,0.627118,1.0
23028,000004.SZ,国华网安,18.2200,801750,5,NaN,None,38.26,18.2200,9.097724e+07,...,55.937084,-0.333333,-0.159305,41.248223,0.329918,16.117309,0.857143,-2.87,1.438119,1.0
106,000004.SZ,国华网安,18.2900,801750,5,NaN,None,38.26,18.2858,9.097724e+07,...,35.331423,-0.365591,-0.034525,42.523028,0.442105,15.526606,0.571429,-1.67,0.729322,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13916,688687.SH,凯因科技,22.4700,,9,5.0,None,39.71,2.2000,2.912839e+08,...,59.143017,0.509872,0.154785,71.245354,0.698975,18.252788,0.571429,2.07,0.941505,63.0
20824,688687.SH,凯因科技,22.4700,,9,5.0,None,39.71,2.2000,2.912839e+08,...,63.861656,-0.043928,-1.195035,57.540808,-0.115741,18.147297,0.714286,0.25,1.098060,63.0
11090,688687.SH,凯因科技,22.4700,801150,9,5.0,None,39.71,2.2000,2.912839e+08,...,59.378478,0.653710,1.044579,66.764319,0.692063,17.245371,0.428571,3.57,1.983133,63.0
24603,688687.SH,凯因科技,22.4700,801150,9,5.0,None,39.71,2.2000,2.912839e+08,...,58.712986,0.470588,2.062002,72.649189,0.422827,20.274861,0.857143,6.32,2.828654,63.0
